# Imports and Connections

In [1]:
import sys
import pandas as pd
import numpy as np
import urllib
import sqlalchemy
import string
import datetime

In [2]:
#Import shared functions
sys.path.append('..\..')
from IPM_Shared_Code_public.Python.utils import get_config

In [3]:
config = get_config('c:\Projects\config.ini')

driver = config['srv']['driver']
server = config['srv']['dev_server']
sladb = config['db']['sladb']

In [4]:
con_string = 'Driver={' + driver + '};Server=' + server +';Database=' + sladb + ';Trusted_Connection=Yes;'
params = urllib.parse.quote_plus(con_string)
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

## Define TODAY

In [55]:
today = datetime.datetime.today()

In [56]:
timing = []

In [57]:
descriptions = []

In [58]:
descriptions_nontech = []

In [59]:
results = []

# Write all of the sql queries required for the tests, execute, and read into pandas dfs:

## tbl_sla_season

In [7]:
sql_sla_season = 'select * from sladb.dbo.tbl_sla_season'

In [8]:
before_sla_season = pd.read_sql(con = engine, sql = sql_sla_season)

In [9]:
date_cols_season = [
    'effective_start', 'effective_end', 'created_date_utc', 'updated_date_utc',
    'effective_start_adj', 'effective_end_adj'
]

In [10]:
for i in date_cols_season:
    before_sla_season[i] = pd.to_datetime(before_sla_season[i])

In [11]:
before_sla_season.head()

,season_id,season_desc,year_round,effective,effective_start,effective_start_adj,effective_end,effective_end_adj,created_date_utc,updated_date_utc
0,1,"Year-round, not seasonal",True,True,2014-01-01,2014-01-05,NaT,NaT,2020-09-17 20:42:13.240,2020-09-17 20:42:13.287
1,2,"Beaches, etc.",False,True,2019-07-01,2019-07-07,NaT,NaT,2020-09-17 20:42:13.427,2020-09-17 20:42:13.427
2,3,"Ballfields, etc.",False,True,2019-07-01,2019-07-07,NaT,NaT,2020-09-17 20:42:13.520,2020-09-17 20:42:13.520


### tbl_sla_season_date

In [12]:
sql_sla_season_date = 'select * from tbl_sla_season_date'

In [13]:
sla_season_date = pd.read_sql(sql = sql_sla_season_date, con = engine)

In [14]:
# sla_season_date.head()

In [15]:
date_cols_season_date = [
    'effective_start', 'effective_end', 'effective_start_adj',
    'effective_end_adj'
]

In [16]:
for i in date_cols_season_date:
    sla_season_date[i] = pd.to_datetime(sla_season_date[i])

## vw_sla_historic

In [17]:
sql_view_historic = 'select * from vw_sla_historic'

In [18]:
view_historic = pd.read_sql(con=engine, sql = sql_view_historic)

In [19]:
date_cols_view_historic = ['effective_start_adj', 'effective_end_adj']

In [20]:
for i in date_cols_view_historic:
    view_historic[i] = pd.to_datetime(view_historic[i])

### vw_sla_code_pivot

In [21]:
sql_vw_sla_code_pivot = 'select * from vw_sla_code_pivot'

In [22]:
before_sla_code_pivot = pd.read_sql(con = engine, sql = sql_vw_sla_code_pivot)

In [23]:
# before_sla_code_pivot.head()

## tbl_ref_unit

In [24]:
sql_ref_unit = 'select * from tbl_ref_unit'

In [25]:
ref_unit_tbl  = pd.read_sql(con = engine, sql = sql_ref_unit)

In [26]:
ref_unit_tbl.head()

,unit_id,unit_desc,unit_class,unit_mrc,unit_status,gisobjid,unit_commiss,unit_updated,unit_withdraw
0,-,Fordham Bedford Lot Busters,PARK,M-11,D,100004689.0,2010-01-06,2018-12-20 12:40:52.483,2010-04-01
1,11772,Fordham Bedford Lot Busters,PARK,M-11,D,NaN,2011-07-28,2018-12-20 12:40:52.810,2014-04-11
2,12654,Fordham Bedford Lot Busters,PARK,M-11,D,NaN,2012-03-06,2018-12-20 12:40:52.983,2014-04-11
3,14301,Fordham Bedford Lot Busters,PARK,M-11,D,NaN,2013-06-03,2018-12-20 12:40:53.150,2014-04-11
4,14918,ZONE,ZONE,M-11,D,100008685.0,2015-02-19,2018-12-20 12:41:07.357,2015-02-19


## tbl_ref_calendar

In [27]:
sql_ref_calendar = 'select * from tbl_ref_calendar'

In [28]:
calendar_tbl  = pd.read_sql(con = engine, sql = sql_ref_calendar)

In [29]:
date_cols_calendar = ['ref_date']

In [30]:
for i in date_cols_calendar:
    calendar_tbl[i] = pd.to_datetime(calendar_tbl[i])

In [31]:
calendar_tbl.head()

,ref_date,ref_year,month_name_desc,day_number,day_name_desc,day_rank_id
0,2014-01-01,2014,January,1,Wednesday,1
1,2014-01-02,2014,January,2,Thursday,1
2,2014-01-03,2014,January,3,Friday,1
3,2014-01-04,2014,January,4,Saturday,1
4,2014-01-05,2014,January,5,Sunday,1


## tbl_ref_sla_season_day_name

In [32]:
sql_day_name = 'select * from tbl_ref_sla_season_day_name'

In [33]:
day_name_tbl  = pd.read_sql(con = engine, sql = sql_day_name)
day_name_tbl.head()

,day_name_num,day_name_desc,ndays_sunday,ndays_saturday
0,6,Friday,2,1
1,2,Monday,6,5
2,7,Saturday,1,0
3,1,Sunday,0,6
4,5,Thursday,3,2


## tbl_change_request

In [34]:
sql_change_request = 'select * from sladb.dbo.tbl_change_request'

In [35]:
before_change_request = pd.read_sql(con = engine, sql = sql_change_request)

In [36]:
before_change_request.head()

,change_request_id,unit_id,sla_code,season_id,effective_start,effective_start_adj,change_request_justification,change_request_comments,sla_change_status
0,1,RZ245,11,1,2014-01-01,2014-01-05,Original Review,None,2
1,2,RZ25,11,1,2014-01-01,2014-01-05,Original Review,None,2
2,3,RZ255,11,1,2014-01-01,2014-01-05,Original Review,None,2
3,4,RZ258,11,1,2014-01-01,2014-01-05,Original Review,None,2
4,5,RZ26,11,1,2014-01-01,2014-01-05,Original Review,None,2


In [37]:
date_cols_change_req = ['effective_start', 'effective_start_adj']

In [38]:
for i in date_cols_change_req:
    before_change_request[i] = pd.to_datetime(before_change_request[i])

## tbl_change_request_status

In [39]:
sql_change_request_status = 'select * from sladb.dbo.tbl_change_request_status'

In [40]:
before_change_request_status = pd.read_sql(con = engine, sql = sql_change_request_status)

In [41]:
before_change_request_status.head()

,change_request_status_id,change_request_id,sla_change_status,created_date_utc,created_user
0,1,3058,1,2014-01-01,0000000
1,2,3057,1,2014-01-01,0000000
2,3,3056,1,2014-01-01,0000000
3,4,3055,1,2014-01-01,0000000
4,5,3054,1,2014-01-01,0000000


In [42]:
date_cols_change_req_stat = ['created_date_utc']

In [43]:
for i in date_cols_change_req_stat:
    before_change_request_status[i] = pd.to_datetime(before_change_request_status[i])

## tbl_unit_sla_season

In [44]:
sql_unit_sla_season = 'select * from sladb.dbo.tbl_unit_sla_season'

In [45]:
before_unit_season = pd.read_sql(con = engine, sql = sql_unit_sla_season)
before_unit_season.head()

,sla_season_id,unit_id,sla_code,season_id,effective,effective_start_adj,effective_end_adj,change_request_id,created_date_utc,updated_date_utc
0,1,RZ245,11,1,False,2014-01-05,2020-03-21,1,2020-09-17 20:42:16.020,2020-09-17 20:42:20.503
1,2,RZ25,11,1,False,2014-01-05,2020-03-21,2,2020-09-17 20:42:16.020,2020-09-17 20:42:20.503
2,3,RZ255,11,1,False,2014-01-05,2020-03-21,3,2020-09-17 20:42:16.020,2020-09-17 20:42:20.503
3,4,RZ258,11,1,False,2014-01-05,2020-03-21,4,2020-09-17 20:42:16.020,2020-09-17 20:42:20.503
4,5,RZ26,11,1,False,2014-01-05,2020-03-21,5,2020-09-17 20:42:16.020,2020-09-17 20:42:20.503


In [46]:
date_cols_unit_season = [
      'created_date_utc', 'updated_date_utc',
    'effective_start_adj', 'effective_end_adj'
]

In [47]:
for i in date_cols_unit_season:
    before_unit_season[i] = pd.to_datetime(before_unit_season[i])

In [50]:
def strip_white_space(row, col):
    return row[col].strip()

In [51]:
before_unit_season['unit_id'] = before_unit_season.apply(lambda row: strip_white_space(row,'unit_id'), axis=1)

# Before Tests

## First before test

In [60]:
timing.append('before')

In [61]:
descriptions.append('''In tbl_unit_change_request test to make sure \
the 'effective_start_adj' is between 'effective_start_adj' and \
'effective_end_adj' from tbl_sla_season if not null. If 'effective_end_adj' \
is null in tbl_sla_season, replace with today's date+1year. If this \
condition is not met, 'sla_change_status' must == 4''')

In [62]:
descriptions_nontech.append('''Make sure that data ranges in the unit table \
have valid start/end dates - i.e. they correspond to season definitions in \
the sla_season table. Otherwise, their status should be 'Invalid' ''')

In [63]:
temp_df = before_change_request.merge(before_sla_season[[
    'season_id', 'effective_start_adj', 'effective_end_adj'
]],
                                      on='season_id')[[
                                          'unit_id', 'season_id',
                                          'effective_start_adj_x',
                                          'effective_start_adj_y',
                                          'effective_end_adj',
                                          'sla_change_status'
                                      ]].copy()
temp_df.head()

,unit_id,season_id,effective_start_adj_x,effective_start_adj_y,effective_end_adj,sla_change_status
0,RZ245,1,2014-01-05,2014-01-05,NaT,2
1,RZ25,1,2014-01-05,2014-01-05,NaT,2
2,RZ255,1,2014-01-05,2014-01-05,NaT,2
3,RZ258,1,2014-01-05,2014-01-05,NaT,2
4,RZ26,1,2014-01-05,2014-01-05,NaT,2


In [64]:
temp_df['effective_end_adj'].fillna(datetime.datetime(
            today.year + 1, today.month, today.day), inplace=True)

In [65]:
temp_df2 = temp_df[
        (temp_df['effective_start_adj_x'] < temp_df['effective_start_adj_y'])
        | (temp_df['effective_start_adj_x'] > temp_df['effective_end_adj'])]

In [66]:
results.append(
    len(temp_df2[temp_df2['sla_change_status']!=4])
    == 0)

In [67]:
temp_df2[temp_df2['sla_change_status']!=4]

,unit_id,season_id,effective_start_adj_x,effective_start_adj_y,effective_end_adj,sla_change_status


## Second before test

In [68]:
timing.append('before')

In [69]:
descriptions.append('''In tbl_unit_sla_season test that \
'effective_start_adj' and 'effective_end_adj' are always \
Sundays and Saturdays, respectively''')

In [70]:
temp_df1 = before_sla_season.merge(calendar_tbl,
                        left_on='effective_start_adj',
                        right_on='ref_date')

In [71]:
temp_df2 = before_sla_season.merge(calendar_tbl,
                        left_on='effective_end_adj',
                        right_on='ref_date')

In [72]:
results.append(len(temp_df1[temp_df1['day_name_desc'] != 'Sunday']) == 0 & len(
    temp_df2[temp_df2['day_name_desc'] != 'Saturday']) == 0)

## Third before test

In [73]:
timing.append('before')

In [74]:
descriptions.append('''In tbl_change_request test that for any \
change request that has 'sla_change_status'==2 the \
'effective_start_adj' is the Sunday after the 'effective_start' IF \
'date_created_utc' (in tbl_change_request_status) <= Sunday after \
'effective_start'. Otherwise, 'effective_start_adj' should be \
the Sunday after 'date_created_utc' ''')

In [75]:
def get_date_from_datetime(row,col):
    return datetime.datetime(row[col].year,
                             row[col].month,
                             row[col].day)    

In [76]:
before_change_request_status[
    'created_date_utc'] = before_change_request_status.apply(
        lambda row: get_date_from_datetime(row, 'created_date_utc'), axis=1)

In [77]:
temp_df = before_change_request_status[
    before_change_request_status['sla_change_status'] == 2].merge(
        before_change_request,how = 'right', on='change_request_id')[[
            'change_request_id', 'effective_start', 'effective_start_adj',
            'created_date_utc', 'sla_change_status_x', 'sla_change_status_y'
        ]].merge(calendar_tbl,how = 'left', left_on='effective_start',
                 right_on='ref_date').merge(day_name_tbl, on='day_name_desc')[[
                     'change_request_id', 'effective_start',
                     'effective_start_adj', 'created_date_utc',
                     'day_name_desc', 'ndays_sunday'
                 ]]

In [78]:
temp_df2 = before_change_request_status[
    before_change_request_status['sla_change_status'] == 2].merge(
        before_change_request, how='right', on='change_request_id')[[
            'change_request_id', 'effective_start', 'effective_start_adj',
            'created_date_utc', 'sla_change_status_x', 'sla_change_status_y'
        ]].merge(calendar_tbl, left_on='created_date_utc',
                 right_on='ref_date').merge(day_name_tbl, on='day_name_desc')[[
                     'change_request_id', 'effective_start',
                     'effective_start_adj', 'created_date_utc',
                     'day_name_desc', 'ndays_sunday'
                 ]]

In [79]:
def get_sunday_after(row, col):
    return datetime.datetime(row[col].year,
                             row[col].month,
                             row[col].day) + datetime.timedelta(
                                 days=row['ndays_sunday'])

In [80]:
temp_df['sunday_after_effstart'] = temp_df.apply(
    lambda row: get_sunday_after(row, 'effective_start'), axis=1)

In [81]:
temp_df['sunday_after_created_date'] = temp_df2.apply(
    lambda row: get_sunday_after(row, 'created_date_utc'), axis=1)

In [82]:
results.append((len(temp_df[(temp_df['created_date_utc']<=temp_df['sunday_after_effstart']) &
       (temp_df['effective_start_adj']!=temp_df['sunday_after_effstart'])])==0
&
len(temp_df[(temp_df['created_date_utc']>temp_df['sunday_after_effstart']) &
       (temp_df['effective_start_adj']!=temp_df['sunday_after_created_date'])])==0))

## Fourth before test

In [85]:
timing.append('before')

In [86]:
descriptions.append('''In tbl_change_request test \
that each unit only has one entry with 'sla_change_status'==1''')

In [87]:
# before_change_request.head()

In [88]:
# temp_df = before_change_request.groupby('unit_id').count()
# units_mult_stats = temp_df[temp_df['sla_change_status']>1].index.values

In [89]:
# temp_df2 = before_change_request[before_change_request['unit_id'].isin(units_mult_stats)][
#     ['unit_id', 'sla_change_status']].sort_values(by='unit_id')

In [90]:
temp_df = before_change_request[before_change_request['sla_change_status'] ==
                                1].groupby('unit_id').count()

In [91]:
results.append(len(temp_df[temp_df['sla_change_status']>1])==0)

## Fifth before test

In [92]:
timing.append('before')

In [93]:
descriptions.append('''In tbl_change_request test that each entry has \
a unique combination 'unit_id', 'effective_start_adj', and 'sla_change_status' \
fields''')

In [94]:
# before_change_request.head()

In [95]:
# before_unit_season.head()

In [96]:
temp_df = before_change_request.groupby(
    ['unit_id', 'effective_start_adj', 'sla_change_status']).count()

In [97]:
# len(temp_df[temp_df['sla_code']>1])==0

In [98]:
results.append(len(temp_df[temp_df['sla_code']>1])==0)

## Sixth before test

In [99]:
timing.append('before')

In [100]:
descriptions.append('''In tbl_unit_sla_season test that each entry has \
a unique combination 'unit_id' and 'effective_start_adj' ''')

In [101]:
# before_change_request.head()

In [102]:
# before_unit_season.head()

In [103]:
temp_df = before_unit_season.groupby(
    ['unit_id', 'effective_start_adj']).count()

In [104]:
len(temp_df[temp_df['sla_code']>1])==0

True

In [105]:
results.append(len(temp_df[temp_df['sla_code']>1])==0)

## Seventh before test

In [106]:
timing.append('before')

In [107]:
descriptions.append('''In tbl_unit_sla_season test there are no overlaps \
in dates ('effective_start_adj' and 'effective_end_adj')''')

In [108]:
sql_overlaps = '''
select l.*
from sladb.dbo.tbl_unit_sla_season as l
left join   
     sladb.dbo.tbl_unit_sla_season as r
on l.unit_id = r.unit_id
where l.effective_start_adj between r.effective_start_adj and 
coalesce(r.effective_end_adj, cast(getdate() as date)) or
coalesce(l.effective_end_adj, cast(getdate() as date)) between 
r.effective_start_adj and coalesce(r.effective_end_adj, cast(getdate() as date))
'''

In [109]:
overlaps_df = pd.read_sql(con = engine, sql = sql_overlaps)

In [110]:
results.append(len(before_unit_season)==len(overlaps_df))

## Eighth before test

In [118]:
timing.append('before')

In [119]:
descriptions.append('''Validate that change requests have consistent 'year_round' \
values in tbl_sla_season and vw_sla_code_pivot''')

In [120]:
sql_multi_join = '''
SELECT l.sla_code
      ,l.[season_id]
      ,l.sla_change_status
      ,r.year_round   as yr_round_season
      ,r2.year_round  as yr_round_sla_code
  FROM [sladb].[dbo].[tbl_change_request] as  l
join
(SELECT season_id
      ,year_round
  FROM [sladb].[dbo].[tbl_sla_season]) as  r 
  on l.season_id = r.season_id 
  join 
  [sladb].[dbo].[vw_sla_code_pivot] as r2
  on l.sla_code = r2.sla_code
'''

In [121]:
multi_join = pd.read_sql(con = engine, sql = sql_multi_join)

In [122]:
multi_join[multi_join['yr_round_season']!=multi_join['yr_round_sla_code']]

,sla_code,season_id,sla_change_status,yr_round_season,yr_round_sla_code


In [123]:
results.append(
    len(multi_join[multi_join['yr_round_season']!=multi_join['yr_round_sla_code']])==0)

## Ninth before test

In [124]:
timing.append('before')

In [125]:
descriptions.append('''Compare tbl_ref_unit and tbl_change_request and test \
that 'effective_start' <= 'unit_withdraw' for all change requests where \
'sla_change_status' !=4 ''')

In [126]:
# ref_unit_tbl['unit_withdraw'].fillna(datetime.datetime(
#             today.year + 1, today.month, today.day),inplace=True)

In [127]:
temp_df = before_change_request[before_change_request['sla_change_status'] != 4].merge(
    ref_unit_tbl, on='unit_id')[['effective_start','effective_start_adj','unit_withdraw']]

In [128]:
temp_df[temp_df['effective_start']>temp_df['unit_withdraw']]

,effective_start,effective_start_adj,unit_withdraw


In [129]:
results.append(len(temp_df[temp_df['effective_start']>temp_df['unit_withdraw']])==0)

## Tenth before test

In [130]:
timing.append('before')

In [131]:
descriptions.append('''Compare tbl_unit_sla_season and tbl_ref_unit \
and test that 'effective_end_adj' == Saturday after 'unit_withdraw' \
and that 'effective' == 0  ''')

In [132]:
def get_saturday_after(row, col):
    if pd.notnull(row[col]):
        return datetime.datetime(
            row[col].year, row[col].month,
            row[col].day) + datetime.timedelta(days=row['ndays_saturday'])
    else:
        return row[col]

In [133]:
temp_df = before_unit_season.merge(ref_unit_tbl, on='unit_id')[[
    'unit_id', 'effective_start_adj', 'effective_end_adj', 'unit_withdraw',
    'effective'
]].merge(calendar_tbl[['ref_date', 'day_name_desc']],
         left_on='effective_end_adj',
         right_on='ref_date').merge(
             day_name_tbl[['day_name_desc', 'ndays_saturday']],
             on='day_name_desc')

In [134]:
temp_df['sat_after_effendadj']=temp_df.apply(lambda row: get_saturday_after(row, 'effective_end_adj'),axis=1)

In [136]:
temp_df['sat_after_unitwithdraw']=temp_df.apply(lambda row: get_saturday_after(row, 'unit_withdraw') ,axis=1)

In [137]:
# temp_df['sat_after_unitwithdraw'] = temp_df.apply(
#     lambda row: get_saturday_after(row, 'unit_withdraw')
#     if (np.all(pd.notnull(row['unit_withdraw'][1]))) else row['unit_withdraw'],
#     axis=1)

In [138]:
results.append(
    len(temp_df[
        temp_df['effective_end_adj'] != temp_df['sat_after_effendadj']]) == 0)

In [139]:
temp_df[pd.notnull(temp_df['unit_withdraw'])]

,unit_id,effective_start_adj,effective_end_adj,unit_withdraw,effective,ref_date,day_name_desc,ndays_saturday,sat_after_effendadj,sat_after_unitwithdraw
6394,X128-XZ475,2014-01-05,2016-02-06,2016-02-01,False,2016-02-06,Saturday,0,2016-02-06,2016-02-01
6395,M071-02,2014-01-05,2017-04-15,2017-04-13,False,2017-04-15,Saturday,0,2017-04-15,2017-04-13
6396,M071-ZN31,2014-01-05,2017-04-15,2017-04-13,False,2017-04-15,Saturday,0,2017-04-15,2017-04-13
6397,M071-ZN32,2014-01-05,2017-04-15,2017-04-13,False,2017-04-15,Saturday,0,2017-04-15,2017-04-13
6398,R065-ZN10,2019-07-07,2019-09-21,2019-09-18,False,2019-09-21,Saturday,0,2019-09-21,2019-09-18
6399,Q448-ZN01,2019-07-07,2019-09-21,2019-09-18,False,2019-09-21,Saturday,0,2019-09-21,2019-09-18
6400,Q448-ZN02,2019-07-07,2019-09-21,2019-09-18,False,2019-09-21,Saturday,0,2019-09-21,2019-09-18
6401,RZ243,2019-07-07,2020-08-29,2020-08-28,False,2020-08-29,Saturday,0,2020-08-29,2020-08-28
6402,X002-10,2019-07-07,2020-08-29,2020-08-28,False,2020-08-29,Saturday,0,2020-08-29,2020-08-28
6403,Q135-01,2019-07-07,2020-08-29,2020-08-28,False,2020-08-29,Saturday,0,2020-08-29,2020-08-28


In [140]:
temp_df.head()

,unit_id,effective_start_adj,effective_end_adj,unit_withdraw,effective,ref_date,day_name_desc,ndays_saturday,sat_after_effendadj,sat_after_unitwithdraw
0,RZ245,2014-01-05,2020-03-21,NaT,False,2020-03-21,Saturday,0,2020-03-21,NaT
1,RZ25,2014-01-05,2020-03-21,NaT,False,2020-03-21,Saturday,0,2020-03-21,NaT
2,RZ255,2014-01-05,2020-03-21,NaT,False,2020-03-21,Saturday,0,2020-03-21,NaT
3,RZ258,2014-01-05,2020-03-21,NaT,False,2020-03-21,Saturday,0,2020-03-21,NaT
4,RZ26,2014-01-05,2020-03-21,NaT,False,2020-03-21,Saturday,0,2020-03-21,NaT


In [141]:
# calendar_tbl.head()

In [142]:
# results.append(
#     len(temp_df[(temp_df['effective_end'] < temp_df['unit_withdraw'])
#             & temp_df['effective'] == True]) == 0)

### Eleventh before test

In [143]:
timing.append('before')

In [144]:
descriptions.append('''Test that for every record in tbl_change_request there \
is at least one record in tbl_change_request_status and that the first one has \
'sla_change_status' ==1''')

In [145]:
temp_df = before_change_request.merge(before_change_request_status,
                            on='change_request_id').sort_values(
                                by=['change_request_id', 'created_date_utc'
                                    ]).groupby('change_request_id').first()


In [146]:
results.append(
    (len(temp_df)==len(before_change_request))&len(temp_df[temp_df['sla_change_status_y']!=1])==0)

### Twelfth before test

In [147]:
timing.append('before')

In [148]:
descriptions.append('''Test that len(tbl_change_request) >= \
len(tbl_unit_sla_season)''')

In [149]:
results.append(len(before_unit_season)<=len(before_change_request))

### Thirteenth before test

In [150]:
timing.append('before')

In [151]:
descriptions.append('''In tbl_change_request_status, if there are entries \
with 'sla_change_status' == 4 make sure those entries have \
'created_date_utc'== max('created_date_utc') for that change request''')

In [152]:
temp_df = before_change_request_status[
    before_change_request_status['sla_change_status'] == 4][[
        'change_request_id', 'sla_change_status', 'created_date_utc'
    ]].sort_values(by='change_request_id').merge(
        before_change_request_status.groupby('change_request_id')[[
            'change_request_id', 'sla_change_status', 'created_date_utc'
        ]].max()[['sla_change_status', 'created_date_utc']],
        left_on='change_request_id',
        right_index=True,
        how='left')
temp_df.head()

,change_request_id,sla_change_status_x,created_date_utc_x,sla_change_status_y,created_date_utc_y


In [153]:
len(temp_df[
        (temp_df['sla_change_status_x'] != temp_df['sla_change_status_y'])
        | (temp_df['created_date_utc_x'] != temp_df['created_date_utc_y'])]) ==0

True

In [154]:
results.append(
    len(temp_df[
        (temp_df['sla_change_status_x'] != temp_df['sla_change_status_y'])
        | (temp_df['created_date_utc_x'] != temp_df['created_date_utc_y'])]) ==
    0)

In [155]:
before_change_request_status[[
    'change_request_id', 'sla_change_status', 'created_date_utc'
]].groupby('change_request_id').max().reset_index()

,change_request_id,sla_change_status,created_date_utc
0,1,2,2014-01-01
1,2,2,2014-01-01
2,3,2,2014-01-01
3,4,2,2014-01-01
4,5,2,2014-01-01
...,...,...,...
11390,11391,2,2020-07-05
11391,11392,2,2020-07-05
11392,11393,2,2020-07-05
11393,11394,2,2020-07-05


### Fourteenth before test

In [156]:
timing.append('before')

In [157]:
descriptions.append('''Test that the latest entry in \
tbl_change_request_status for each 'change_request_id' is the same \
as the entry in tbl_change_request for that change request''')

In [158]:
results.append(len(before_change_request_status[[
    'change_request_id', 'sla_change_status', 'created_date_utc'
]].groupby('change_request_id').max().reset_index().merge(
    before_change_request[['change_request_id', 'sla_change_status'
                           ]].sort_values(by='change_request_id'),
    on=['change_request_id', 'sla_change_status'],
    how='left'))==len(before_change_request))

In [159]:
# len(before_change_request)

In [160]:
# before_change_request_status[[
#     'change_request_id', 'sla_change_status', 'created_date_utc'
# ]].sort_values(by='change_request_id').merge(before_change_request_status[[
#     'change_request_id', 'sla_change_status', 'created_date_utc'
# ]].groupby('change_request_id').max().reset_index(),
#                                              left_on=['change_request_id','sla_change_status_x'],
#                                              right_on = ['change_request_id', 'sla_change_status_y']
                                             
# #                                              right_index=True,
#                                              )

### Fifteenth before test

In [161]:
timing.append('before')

In [162]:
descriptions.append('''Test to make sure there are no gaps in dates \
between entries in the view used for reporting''')

In [163]:
# view_historic.head()

In [164]:
# view_historic[~pd.isnull(view_historic['effective_start_adj'])].groupby(
#     'unit_id').min()[['effective_start_adj']]

In [165]:
# view_historic[view_historic['unit_id']=='B382A']

In [166]:
# view_historic[view_historic['unit_id']=='Q021-ZN04']

In [167]:
sql_gaps_1 = '''
select unit_id,
       datediff(day, min(effective_start_adj), max(effective_end_adj)) as daydiff
from sladb.dbo.vw_sla_historic
where sla_id is not null
group by unit_id

'''

sql_gaps_2 = '''select unit_id,
       case when count(*) > 1 then sum(daydiff) + (count(*) - 1)
            else sum(daydiff)
       end as daydiff
from (select unit_id,
             datediff(day, effective_start_adj, effective_end_adj) as daydiff
      from sladb.dbo.vw_sla_historic 
      where sla_id is not null) as t
group by unit_id
'''

In [168]:
gaps_df1 = pd.read_sql(con = engine, sql = sql_gaps_1)
gaps_df2 = pd.read_sql(con = engine, sql = sql_gaps_2)

In [169]:
temp_df = gaps_df1.merge(gaps_df2, on='unit_id')

In [170]:
results.append(len(temp_df)==len(temp_df[temp_df['daydiff_x']==temp_df['daydiff_y']]))

In [171]:
# temp_df[temp_df['daydiff_x']==temp_df['daydiff_y']]

In [172]:
# temp_df

In [173]:
# temp_df[pd.isnull(temp_df['daydiff_y'])]

In [174]:
# results.append(len(temp_df)==len(temp_df[temp_df['daydiff_x']==temp_df['daydiff_y']]))

In [175]:
# season_date.head()

### Sixteenth before test

In [176]:
timing.append('before')

In [177]:
descriptions.append('''Test that range of dates included in \
vw_sla_historic is the same as the range in tbl_unit_sla_season''')

In [178]:
before_unit_season['effective_end_adj'].fillna(datetime.datetime(
            today.year , today.month, today.day), inplace=True)

In [179]:
temp_df = view_historic[[
    'unit_id', 'effective_start_adj', 'effective_end_adj'
]].merge(before_unit_season[[
    'unit_id', 'effective_start_adj', 'effective_end_adj'
]],
         on='unit_id')
temp_df.head()

,unit_id,effective_start_adj_x,effective_end_adj_x,effective_start_adj_y,effective_end_adj_y
0,RZ245,2014-01-05,2020-03-21,2014-01-05,2020-03-21
1,RZ245,2014-01-05,2020-03-21,2020-03-22,2020-07-04
2,RZ245,2014-01-05,2020-03-21,2020-07-05,2020-10-07
3,RZ245,2020-03-22,2020-07-04,2014-01-05,2020-03-21
4,RZ245,2020-03-22,2020-07-04,2020-03-22,2020-07-04


In [180]:
min_temp_df = temp_df.groupby('unit_id').min()
max_temp_df = temp_df.groupby('unit_id').max()

In [181]:
# min_temp_df.head()

In [182]:
results.append((len(min_temp_df[min_temp_df['effective_start_adj_x'] !=
                                min_temp_df['effective_start_adj_y']]) == 0)
               & (len(max_temp_df[max_temp_df['effective_end_adj_x'] !=
                                  max_temp_df['effective_end_adj_y']]) == 0))

In [183]:
# len(max_temp_df[max_temp_df['effective_end_adj_x']!=max_temp_df['effective_end_adj_y']])==0

In [184]:
# before_unit_season[before_unit_season['unit_id']=='M280'][['effective_start_adj','effective_end_adj']]

In [185]:
# view_historic[view_historic['unit_id']=='M280']#[['effective_start_adj','effective_end_adj']]

In [186]:
# min_temp_df.head()

In [187]:
# before_unit_season[(before_unit_season['unit_id']=='X039-ZN14')&(before_unit_season['season_id']==2)]

In [188]:
# temp_df[(temp_df['unit_id']=='X039-ZN14')&(temp_df['season_id']==2)]

In [189]:
# temp_df.groupby('unit_id').count()

In [190]:
# before_unit_season.head()

## Make Updates

## Execute all of the sql queries AFTER UPDATES and store the results

## List of tuples with final test results

In [191]:
list(zip(timing, descriptions, results))

[('before',
  "In tbl_unit_change_request test to make sure the 'effective_start_adj' is between 'effective_start_adj' and 'effective_end_adj' from tbl_sla_season if not null. If 'effective_end_adj' is null in tbl_sla_season, replace with today's date+1year. If this condition is not met, 'sla_change_status' must == 4",
  True),
 ('before',
  "In tbl_unit_sla_season test that 'effective_start_adj' and 'effective_end_adj' are always Sundays and Saturdays, respectively",
  True),
 ('before',
  "In tbl_change_request test that for any change request that has 'sla_change_status'==2 the 'effective_start_adj' is the Sunday after the 'effective_start' IF 'date_created_utc' (in tbl_change_request_status) <= Sunday after 'effective_start'. Otherwise, 'effective_start_adj' should be the Sunday after 'date_created_utc' ",
  True),
 ('before',
  "In tbl_change_request test that each unit only has one entry with 'sla_change_status'==1",
  True),
 ('before',
  "In tbl_change_request test that each en

In [192]:
pd.DataFrame(list(zip(timing, descriptions, results)), columns = {'before/after', 'test_desc', 'result'})             

,result,before/after,test_desc
0,before,In tbl_unit_change_request test to make sure t...,True
1,before,In tbl_unit_sla_season test that 'effective_st...,True
2,before,In tbl_change_request test that for any change...,True
3,before,In tbl_change_request test that each unit only...,True
4,before,In tbl_change_request test that each entry has...,True
5,before,In tbl_unit_sla_season test that each entry ha...,True
6,before,In tbl_unit_sla_season test there are no overl...,True
7,before,Validate that change requests have consistent ...,True
8,before,Compare tbl_ref_unit and tbl_change_request an...,True
9,before,Compare tbl_unit_sla_season and tbl_ref_unit a...,True
